# Bayesian Hyperparameter Optimisation

In this notebook we use Bayesian hyperparameter optimisiation to find the optimal hyperparameters for LightGBM. Following the guide here: https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f and the example notebook: https://github.com/WillKoehrsen/hyperparameter-optimization/blob/master/Bayesian%20Hyperparameter%20Optimization%20of%20Gradient%20Boosting%20Machine.ipynb

## Set-up

In [1]:
import pickle
import os

import pandas as pd
import numpy as np
import shap
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Activation, Dropout
import tensorflow as tf
from keras import initializers
import keras
import lightgbm

Using TensorFlow backend.


In [2]:
os.chdir('..')

In [3]:
from src.data.utils import load_datasets, COLUMN_DESCRIPTION_DICT
from src.models.utils import lgbm_feature_reduction

In [4]:
pd.options.display.max_columns = 50

## Load data

Now includes all features

In [5]:
RANDOM_SEED=42

In [6]:
%%time
train, dev, test, test_kaggle = load_datasets(random_seed=RANDOM_SEED)

Finished previous_application!
Finished installments_payments!
Finished pos_cash_balance!
Finished credit_card_balance!
CPU times: user 2min 37s, sys: 1min 27s, total: 4min 5s
Wall time: 4min 18s


In [7]:
train.shape

(184506, 1151)

In [8]:
# Features used in best LGBM:
FINAL_FEATURES =  pickle.load(open('models/lgbm_all_features_reduced_random_FEATURES.pickle', 'rb'))
len(FINAL_FEATURES)

212

In [9]:
train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,...,POS_CASH_balance_SK_DPD_amax_amin_y,POS_CASH_balance_SK_DPD_amax_amax_y,POS_CASH_balance_SK_DPD_DEF_mean_mean_y,POS_CASH_balance_SK_DPD_DEF_mean_sum_y,POS_CASH_balance_SK_DPD_DEF_mean_amin_y,POS_CASH_balance_SK_DPD_DEF_mean_amax_y,POS_CASH_balance_SK_DPD_DEF_sum_mean_y,POS_CASH_balance_SK_DPD_DEF_sum_sum_y,POS_CASH_balance_SK_DPD_DEF_sum_amin_y,POS_CASH_balance_SK_DPD_DEF_sum_amax_y,POS_CASH_balance_SK_DPD_DEF_amin_mean_y,POS_CASH_balance_SK_DPD_DEF_amin_sum_y,POS_CASH_balance_SK_DPD_DEF_amin_amin_y,POS_CASH_balance_SK_DPD_DEF_amin_amax_y,POS_CASH_balance_SK_DPD_DEF_amax_mean_y,POS_CASH_balance_SK_DPD_DEF_amax_sum_y,POS_CASH_balance_SK_DPD_DEF_amax_amin_y,POS_CASH_balance_SK_DPD_DEF_amax_amax_y,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Active_y,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Approved_y,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Completed_y,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Demand_y,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Refused,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Sent_proposal,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Signed_y
170657,297783,0.0,0,74250.0,112500.0,6282.0,112500.0,0.018850,-15077,-7915,-2853.0,-8,NaN,1,1,1,1,0,0,2.0,2,2,10,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0
251121,390572,0.0,2,225000.0,810000.0,40500.0,810000.0,0.035792,-14311,-1858,-2306.0,-5196,4.0,1,1,0,1,0,0,4.0,2,2,13,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96789,212363,0.0,0,225000.0,1546020.0,42642.0,1350000.0,0.032561,-15236,-5751,-7483.0,-4492,8.0,1,1,0,1,1,0,2.0,1,1,16,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0
14042,116368,1.0,1,202500.0,444420.0,30195.0,337500.0,0.018850,-17688,-754,-4307.0,-1243,22.0,1,1,0,1,0,0,3.0,2,2,14,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,1.0,0.0,0.0,0.0,0.0
32597,137783,0.0,2,126000.0,1216201.5,35689.5,1062000.0,0.018029,-14030,-368,-3400.0,-4447,NaN,1,1,1,1,0,0,4.0,3,3,6,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
X_train = train[FINAL_FEATURES]
y_train = train['TARGET']
X_dev = dev[FINAL_FEATURES]
y_dev = dev['TARGET']

In [11]:
lgbm_model_random = pickle.load(open('models/lgbm_all_features_reduced_random.pickle', 'rb'))

Best parameters after 80 iterations

In [12]:
lgbm_model_random.best_params_

{'subsample_for_bin': 200000,
 'subsample': 0.5757575757575758,
 'reg_lambda': 15.388177500383463,
 'reg_alpha': 17.188391428171457,
 'num_leaves': 26,
 'min_child_samples': 175,
 'learning_rate': 0.12312929581752732,
 'colsample_bytree': 0.6444444444444444}

In [13]:
lgbm_model_random.best_score_

0.7809554450455519

In [14]:
predictions = lgbm_model_random.predict_proba(X_dev)[:, 1]
roc_auc_score(y_dev, predictions)

0.7778717131161857

### Bayesian Hyperparameter Optimisation

In [125]:
# Function mapping hyperparameters to a real-valued score
def objective(hyperparameters):
    
    # Hacky workaround for bug in package which saves integer hyperparameters as floats in the final dictionary. We explicitly cast as integers to avoid lightgbm errors.
    for hyper in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        try:
            as_int = int(hyperparameters[hyper])
            hyperparameters[hyper] = as_int
        except:
            continue
    
    # Machine learning model
    lgbm = lightgbm.LGBMClassifier(**hyperparameters)
    
    # Training 
    lgbm.fit(X_train, y_train)
    
    # Making predictions and evaluating
    predictions = lgbm.predict_proba(X_dev)[:,1]
    roc_auc = roc_auc_score(y_dev, predictions)
    
    return -roc_auc  # Has to be a minimisation problem

In [55]:
from hyperopt import hp, fmin, rand, tpe, Trials

__NOTE:__

For sampling integer values from a range use `hp.uniformint('num_leaves', 20, 150)` rather than `hp.choice('num_leaves', list(np.arange(20, 150, dtype=int)))`. The latter causes an unusual error where the loss reported in the trials is different from the loss calculated by the objective function.

Also this saves integer values as floats in the final parameter dictionary. Hence if this dictionary is passed to the objective function it throws `LightGBMError: Parameter num_leaves should be of type int, got "20.0"`. Therefore make sure objective function explicitly casts these parameter values as integers.

In [131]:
space = {
    'num_leaves': hp.uniformint('num_leaves', 20, 150),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.5)),
    'subsample_for_bin': hp.uniformint('subsample_for_bin', 20000, 300000),
    'min_child_samples': hp.uniformint('min_child_samples', 20, 500),
    'reg_alpha': hp.loguniform('reg_alpha', np.log(0.01), np.log(100)),
    'reg_lambda': hp.loguniform('reg_lambda', np.log(0.01), np.log(100)),
    'colsample_bytree': hp.quniform('colsample_bytree', low=0.6, high=1, q=1),
    'subsample': hp.quniform('subsample', low=0.5, high=1, q=1)
}

In [122]:
from hyperopt.pyll.stochastic import sample

In [132]:
sample(space)

{'colsample_bytree': 1.0,
 'learning_rate': 0.05975761234335702,
 'min_child_samples': 147,
 'num_leaves': 60,
 'reg_alpha': 0.3896392142880288,
 'reg_lambda': 2.829492190426517,
 'subsample': 1.0,
 'subsample_for_bin': 292047}

According to this issue the first 20 trials are random: https://github.com/hyperopt/hyperopt/issues/642

We therefore use 100 trials in total.

For more details on `fmin` see: https://github.com/hyperopt/hyperopt/wiki/FMin

In [145]:
%%time
bayes_trials = Trials()

best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=bayes_trials)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

2020-05-24 01:03:48,673 - build_posterior_wrapper took 0.002679 seconds
2020-05-24 01:03:48,674 - TPE using 0 trials


  1%|          | 1/100 [00:09<14:57,  9.07s/trial, best loss: -0.776353398357067]

2020-05-24 01:03:57,744 - build_posterior_wrapper took 0.003189 seconds
2020-05-24 01:03:57,746 - TPE using 1/1 trials with best loss -0.776353


  2%|▏         | 2/100 [00:20<15:54,  9.74s/trial, best loss: -0.776353398357067]

...

 98%|█████████▊| 98/100 [17:26<00:21, 10.82s/trial, best loss: -0.7785313381435932]

2020-05-24 01:21:14,737 - build_posterior_wrapper took 0.003650 seconds
2020-05-24 01:21:14,738 - TPE using 98/98 trials with best loss -0.778531


 99%|█████████▉| 99/100 [17:34<00:10, 10.25s/trial, best loss: -0.7785313381435932]

2020-05-24 01:21:23,672 - build_posterior_wrapper took 0.003245 seconds
2020-05-24 01:21:23,673 - TPE using 99/99 trials with best loss -0.778531


100%|██████████| 100/100 [17:45<00:00, 10.65s/trial, best loss: -0.7785313381435932]
CPU times: user 1h 59min 5s, sys: 2min 6s, total: 2h 1min 12s
Wall time: 17min 45s


In [146]:
best

{'colsample_bytree': 1.0,
 'learning_rate': 0.11273666179639348,
 'min_child_samples': 433.0,
 'num_leaves': 36.0,
 'reg_alpha': 9.500420592781932,
 'reg_lambda': 9.17552610004045,
 'subsample': 1.0,
 'subsample_for_bin': 179406.0}

In [147]:
bayes_trials.average_best_error()

-0.7785313381435932

Check that same error is returned if we apply the best params directly to the objective function (the integer sampling bug was identified this way)

In [148]:
objective(best)

-0.7785313381435932

In [149]:
hyperparameters = best.copy()

for hyper in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
    try:
        as_int = int(hyperparameters[hyper])
        hyperparameters[hyper] = as_int
    except:
        continue

# Machine learning model
lgbm = lightgbm.LGBMClassifier(**hyperparameters)

# Training 
lgbm.fit(X_train, y_train)

# Making predictions and evaluating
predictions = lgbm.predict_proba(X_dev)[:,1]
roc_auc = roc_auc_score(y_dev, predictions)

In [150]:
roc_auc

0.7785313381435932

Compare Bayes params with random search:

In [151]:
best

{'colsample_bytree': 1.0,
 'learning_rate': 0.11273666179639348,
 'min_child_samples': 433,
 'num_leaves': 36,
 'reg_alpha': 9.500420592781932,
 'reg_lambda': 9.17552610004045,
 'subsample': 1.0,
 'subsample_for_bin': 179406}

In [152]:
lgbm_model_random.best_params_

{'subsample_for_bin': 200000,
 'subsample': 0.5757575757575758,
 'reg_lambda': 15.388177500383463,
 'reg_alpha': 17.188391428171457,
 'num_leaves': 26,
 'min_child_samples': 175,
 'learning_rate': 0.12312929581752732,
 'colsample_bytree': 0.6444444444444444}

### Summary

Bayes search finds a set of params which beats random search in just 24 trials!

### Make submission

In [153]:
hyperparameters

{'colsample_bytree': 1.0,
 'learning_rate': 0.11273666179639348,
 'min_child_samples': 433,
 'num_leaves': 36,
 'reg_alpha': 9.500420592781932,
 'reg_lambda': 9.17552610004045,
 'subsample': 1.0,
 'subsample_for_bin': 179406}

In [157]:
lgbm = lightgbm.LGBMClassifier(**hyperparameters)

# Training on train and dev
lgbm.fit(X_train.append(X_dev), y_train.append(y_dev))

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.11273666179639348,
               max_depth=-1, min_child_samples=433, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=36,
               objective=None, random_state=None, reg_alpha=9.500420592781932,
               reg_lambda=9.17552610004045, silent=True, subsample=1.0,
               subsample_for_bin=179406, subsample_freq=0)

In [160]:
roc_auc_score(y_train.append(y_dev), lgbm.predict_proba(X_train.append(X_dev))[:,1])

0.8256912723833909

In [161]:
test_kaggle.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,...,POS_CASH_balance_SK_DPD_amax_amin_y,POS_CASH_balance_SK_DPD_amax_amax_y,POS_CASH_balance_SK_DPD_DEF_mean_mean_y,POS_CASH_balance_SK_DPD_DEF_mean_sum_y,POS_CASH_balance_SK_DPD_DEF_mean_amin_y,POS_CASH_balance_SK_DPD_DEF_mean_amax_y,POS_CASH_balance_SK_DPD_DEF_sum_mean_y,POS_CASH_balance_SK_DPD_DEF_sum_sum_y,POS_CASH_balance_SK_DPD_DEF_sum_amin_y,POS_CASH_balance_SK_DPD_DEF_sum_amax_y,POS_CASH_balance_SK_DPD_DEF_amin_mean_y,POS_CASH_balance_SK_DPD_DEF_amin_sum_y,POS_CASH_balance_SK_DPD_DEF_amin_amin_y,POS_CASH_balance_SK_DPD_DEF_amin_amax_y,POS_CASH_balance_SK_DPD_DEF_amax_mean_y,POS_CASH_balance_SK_DPD_DEF_amax_sum_y,POS_CASH_balance_SK_DPD_DEF_amax_amin_y,POS_CASH_balance_SK_DPD_DEF_amax_amax_y,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Active_y,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Approved_y,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Completed_y,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Demand_y,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Refused,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Sent_proposal,POS_CASH_balance_COUNT_COUNT_NAME_CONTRACT_STATUS_Signed_y
307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,2.0,2,2,18,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,2.0,2,2,9,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,2.0,2,2,14,0,0,...,1.0,1.0,0.010417,0.010417,0.010417,0.010417,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0
307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866,-2000.0,-4208,NaN,1,1,0,1,1,0,4.0,2,2,11,0,0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0
307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191,-4000.0,-4262,16.0,1,1,1,1,0,0,3.0,2,2,5,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
predictions = lgbm.predict_proba(test_kaggle[FINAL_FEATURES])[:,1]

In [163]:
test_kaggle['TARGET'] = predictions

In [164]:
test_kaggle[['SK_ID_CURR', 'TARGET']].to_csv('data/processed/best_lgbm_with_bayes.csv', index=False)

In [165]:
!kaggle competitions submit -c home-credit-default-risk -f data/processed/best_lgbm_with_bayes.csv -m "best_lgbm_with_bayes"

100%|███████████████████████████████████████| 1.26M/1.26M [00:08<00:00, 150kB/s]
Successfully submitted to Home Credit Default Risk

Private score: 0.77566

Public score: 0.77324

Interestingly the model performs worse than the model found using random search. This may be a consequence of using cross validation for random search and a single validation set for Bayes. Therefore our Bayes parameters may be 'overfit' to the dev set. The objective function could be modified to use cross validation at the expense of time to compute.